In [1]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\16-End-to-end-Fine-Tuning'

In [2]:
from srcc.textSummarizer.logging import logger
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: int
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [3]:
from srcc.textSummarizer.constants import *
from srcc.textSummarizer.utils.common import *

class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model=config.model,
            num_train_epochs= params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq 
import torch
from datasets import load_from_disk

c:\Users\Ansh Lulla\VS-Code\MLOps\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-29 20:28:48,953: WARNING: module_wrapper: From c:\Users\Ansh Lulla\VS-Code\MLOps\venv\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2025-06-29 20:28:49,829: INFO: config: PyTorch version 2.7.1 available.]
[2025-06-29 20:28:49,833: INFO: config: TensorFlow version 2.19.0 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model).to(device)
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

        dataset = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,               # where to save model/checkpoints
            num_train_epochs=1,                            # total training epochs
            warmup_steps=500,                              # number of warmup steps for learning rate scheduler
            per_device_train_batch_size=1,                 # batch size per GPU/TPU core/CPU
            weight_decay=0.01,                             # strength of weight decay
            logging_steps=10,                              # log every 10 steps
            evaluation_strategy="steps",                   # evaluation strategy
            eval_steps=500,                                # evaluate every 500 steps
            save_steps=int(1e6),                           # save checkpoint every 1M steps
            gradient_accumulation_steps=16,                # accumulate gradients to simulate larger batch
        )

        trainer = Trainer(model=model, 
                          args=training_args,
                          tokenizer=tokenizer,
                          data_collator=data_collator,
                          train_dataset=dataset["test"],
                          eval_dataset=dataset["validation"])
        
        trainer.train()

        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-fine-tuned"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [9]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-06-29 21:03:39,305: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-29 21:03:39,310: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-29 21:03:39,314: INFO: common: Created Directory at: artifacts]
[2025-06-29 21:03:39,315: INFO: common: Created Directory at: artifacts/model_trainer]


Error while downloading from https://cdn-lfs.hf.co/google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1751214822&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTIxNDgyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=NC%7EkfexFJe3epKVdEf89RU73kWYjnWwwskuFfTY5qZskQGBiNYgU5khsc-60fV14rp4S0dIe4bci3-aqfao0N9399GG%7EOg9V3Ay7rsyaBmjT5DugIYEN6fRUps0WEuHaHJiSllG4%7EM5zRYx5oRWdtgKZ1Usg0-D3mCYcjyZ8Pa%7EooytimzjM8buvnWl8XAnS5s-63V2iGJMJqBLFzxyWxIy-oOrGm27liBvwe356OIUx1rJ7uC4dtAymekkxOZHTMVVTc%7EoHFEiQBnSMKsqL0U9ZDflsoOUi

[2025-06-29 21:09:31,007: WARNING: file_download: Error while downloading from https://cdn-lfs.hf.co/google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1751214822&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTIxNDgyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=NC%7EkfexFJe3epKVdEf89RU73kWYjnWwwskuFfTY5qZskQGBiNYgU5khsc-60fV14rp4S0dIe4bci3-aqfao0N9399GG%7EOg9V3Ay7rsyaBmjT5DugIYEN6fRUps0WEuHaHJiSllG4%7EM5zRYx5oRWdtgKZ1Usg0-D3mCYcjyZ8Pa%7EooytimzjM8buvnWl8XAnS5s-63V2iGJMJqBLFzxyWxIy-oOrGm27liBvwe356OIUx1rJ7

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs.hf.co\', port=443): Max retries exceeded with url: /google/pegasus-cnn_dailymail/cf34e28157605d3ced3bd184820ea2a9d9a30b69f4bb1df4aaa4145d0850290e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1751214822&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTIxNDgyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsL2NmMzRlMjgxNTc2MDVkM2NlZDNiZDE4NDgyMGVhMmE5ZDlhMzBiNjlmNGJiMWRmNGFhYTQxNDVkMDg1MDI5MGU~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=NC~kfexFJe3epKVdEf89RU73kWYjnWwwskuFfTY5qZskQGBiNYgU5khsc-60fV14rp4S0dIe4bci3-aqfao0N9399GG~Og9V3Ay7rsyaBmjT5DugIYEN6fRUps0WEuHaHJiSllG4~M5zRYx5oRWdtgKZ1Usg0-D3mCYcjyZ8Pa~ooytimzjM8buvnWl8XAnS5s-63V2iGJMJqBLFzxyWxIy-oOrGm27liBvwe35